## Лабораторная работа №3. Классификация текстовых данных

Решить задачу классификации текстовых данных. Для векторизации текста использовать модель мешка слов/масштабирование данных при помощи tf-idf/word2vec/Glove, использовать 3 модели машинного обучения (включая нейронные сети). Выбрать лучшую комбинацию и оценить результаты. Показать на примере работу лучшей модели (на входе текст на выходе класс).
Данные взяты отсюда https://github.com/natasha/corus#reference
Скачать датасет https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/interfax.csv.gz

# Импортируем необходимые библиотеки

In [11]:
import pandas as pd
import numpy as np
import warnings

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import scale

from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile

# Данные

In [2]:
# Получим первые 5000 значений из файла
data = pd.read_csv("data/interfax.csv.gz", nrows=5000)
# Посмотрим на полученную таблицу
data.head()

,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
0,2019-04-30 13:26:00,https://www.sport-interfax.ru/659837,www.sport-interfax.ru,Спорт,NaN,Российского чемпиона мира по греко-римской бор...,Москва. 30 апреля. INTERFAX.RU - Чемпион мира ...,-,-,-,-,-,-,-,-,-
1,2019-05-07 12:46:00,https://www.sport-interfax.ru/660425,www.sport-interfax.ru,Спорт,NaN,Первый канал покажет все матчи россиян на ЧМ п...,Москва. 7 мая. INTERFAX.RU - Первый канал объя...,-,-,-,-,-,-,-,-,-
2,2019-05-08 12:57:00,https://www.sport-interfax.ru/660543,www.sport-interfax.ru,Спорт,NaN,Семак выразил надежду на новый контракт Кокори...,Москва. 8 мая. INTERFAX.RU - Главный тренер пе...,-,-,-,-,-,-,-,-,-
3,2019-05-09 00:06:00,https://www.sport-interfax.ru/660618,www.sport-interfax.ru,Спорт,NaN,"""Тоттенхэм"" стал вторым финалистом Лиги чемпионов",В финале он сыграет с другим представителем Ан...,-,-,-,-,-,-,-,-,-
4,2019-05-14 14:32:00,https://www.sport-interfax.ru/661091,www.sport-interfax.ru,Спорт,NaN,Претендовавшего на участие в ОИ скейтбордиста ...,Москва. 14 мая. INTERFAX.RU - Российский скейт...,-,-,-,-,-,-,-,-,-


In [3]:
# Извлечем из таблицы необходимые для обучения данные
texts = data.loc[:, 'text'].values
topics = data.loc[:, 'topics'].values

# Подготовим метки(y) для входных данных

In [4]:
# Создадим словарь со всеми классами, встречающимися в файле, и пронумеруем их
classes = {x:i for i, x in enumerate(data['topics'].unique())}

# print(classes)

In [5]:
# Теперь переведем названия классов в цифровые значения
y = []
for i in topics:
    y.append(classes[i])
    
# print(y)

# Подготовим входные данные (X)

In [6]:
# создадим список с перечнем слов для каждого текста в наборе
X = []
for t in texts:
    text_list = t.split(" ")
    X.append(text_list)
    
# X

In [7]:
# Создаем модель Word2Vec, сохраняем
n_dim = 300
path = get_tmpfile("word2vec.model")
model = Word2Vec(vector_size=n_dim, window=3, min_count=3, workers=4, negative=3)
model.build_vocab(X)
model.save("word2vec.model")

In [8]:
# Обучаем модель на всем X и нам становятся доступны векторные представления слов из X
model = Word2Vec.load("word2vec.model")
model.train(X, total_examples=5000, epochs=50)

(38758012, 51419750)

In [9]:
# Суммируем и нормируем все векторные представления слов из каждого текста 
# и получаем один вектор заданной размерности (300) для обучающего набора 
def buildWordVector(text, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in text:
        try:
            vec += model.wv[word].reshape((1, size))
            count += 1
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [12]:
# Создаем нормированный набор обучающих данных
X = np.concatenate([buildWordVector(z, n_dim) for z in X])
X = scale(X)

In [13]:
X

array([[-0.39638533,  0.92016957, -2.28049897, ..., -4.37301874,
         1.31943326, -0.48027532],
       [-0.99034454, -0.46867793, -1.24322244, ..., -0.08421575,
        -1.27781018,  0.26691041],
       [-0.20031841,  1.06420136,  0.19957478, ...,  0.90487023,
        -0.29313403,  0.66053613],
       ...,
       [ 1.59655022, -0.65899073, -0.89238183, ..., -1.86271768,
         1.9988871 ,  0.43768384],
       [ 0.58442631,  1.31217957, -1.56141872, ..., -0.03656632,
         0.70523244, -1.0001677 ],
       [ 0.0261875 , -0.66193418, -0.29634517, ..., -1.0048168 ,
         1.02994657, -1.32075636]])

# Создадим обучающий и тестовый наборы данных

In [14]:
# Разбиваем выборку на тренировочный и тестовый наборы
seed = 256
test_size = 0.2
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

# Обучим и протестируем наши модели

# RandomForestClassifier

In [15]:
from sklearn.ensemble import RandomForestClassifier

# Создаем модель
forest_model = RandomForestClassifier(n_estimators=20, 
                                      criterion='entropy')

# Обучаем полученную модель
epochs = 5
for epoch in range(epochs):
    forest_model.fit(x_train, y_train)

# Проверяем качество работы модели на тестовой выборке
accuracy = accuracy_score(y_test,forest_model.predict(x_test))
print("Процент правильных ответов:", accuracy * 100)

Процент правильных ответов: 72.5


# LinearSVC

In [16]:
from sklearn.svm import LinearSVC

# Создаем модель
linear_svm = LinearSVC()

 # Обучаем полученную модель
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    epochs = 5
    for epoch in range(epochs):
        linear_svm.fit(x_train, y_train)

# Проверяем качество работы модели на тестовой выборке
accuracy = accuracy_score(y_test, linear_svm.predict(x_test))
print("Процент правильных ответов:", accuracy * 100)

Процент правильных ответов: 78.10000000000001


# MLPClassifier

In [17]:
from sklearn.neural_network import MLPClassifier

# Создаем модель
mlpc = MLPClassifier(hidden_layer_sizes = (900), 
                     batch_size=10, 
                     solver='sgd',
                     learning_rate_init = 0.025)

# Обучаем полученную модель
epochs = 5
for epoch in range(epochs):
    mlpc.fit(x_train, y_train)

# Проверяем качество работы модели на тестовой выборке
accuracy = accuracy_score(y_test, mlpc.predict(x_test))
print("Процент правильных ответов:", accuracy * 100)

Процент правильных ответов: 66.4


In [23]:
# Мы видим,что самый высокий процент правильных ответов у LinearSVC. Посмотрим пример ее работы

y_pred = linear_svm.predict(x_test[0].reshape(1,-1))
y_true = y_test[0]

print(y_pred)
print(y_true)

[2]
2
